In [1]:
import envloader 

from langchain.document_loaders import PyPDFLoader,PyPDFDirectoryLoader
from langchain.document_loaders import WebBaseLoader, TextLoader
from langchain.embeddings import GPT4AllEmbeddings



import bs4
from langchain import hub


from langchain.chat_models import ChatOpenAI
from langchain.llms.openllm import OpenLLM
from langchain.llms.gpt4all import GPT4All


from langchain.document_loaders import WebBaseLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.schema import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma

In [2]:
text = r"D:\P2\Python\llm-proj\random.txt" 
link = "https://en.wikipedia.org/wiki/Breaking_Bad"

In [3]:
loader = TextLoader(text)

In [4]:
#loader = WebBaseLoader(link)

In [5]:
splits = loader.load_and_split()
len(splits)

1

In [6]:
import torch 
import gc 
torch.cuda.empty_cache() 
gc.collect()

0

In [7]:
gpt4all_embd = GPT4AllEmbeddings()

In [8]:
vectorstore = Chroma.from_documents(documents=splits, embedding=GPT4AllEmbeddings() , )

In [9]:
retriever = vectorstore.as_retriever()

prompt = hub.pull("rlm/rag-prompt")
llm = GPT4All(model = r"C:\Users\obsidian\Desktop\mistral-7b-openorca.Q4_0.gguf" , )

In [10]:
rag_chain = (
    {"context": retriever , "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [11]:
queries = [
    "Who is Aniruddh?", #Direct 
    "Who is Walter White?", #Not present 
    "What is capitalism?", #General/Not direct 
    "How is Aniruddh linked with Gary Vee?", #Indirect context 
]

for index, query in enumerate(queries , 1 ):
    print(f"Query {index}: {query}")
    print(f"Result: {rag_chain.invoke(query)}\n\n")

Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1


Query 1: Who is Aniruddh?


Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1


Result:  Aniruddh is the former CR head at E-Cell who served between 2021 and 2022, working alongside Yogendra Pratap Singh as a co-manager. He is friends with Shivansh Singh and attempted to launch an NFT for E-Cell which was Asia's first official student body NFT on January 7th, 2022. Aniruddh also tried inviting Gary Vee but ended up being involved in the VeeKids scam.


Query 2: Who is Walter White?


Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1


Result:  I don't know who Walter White is based on the given context.


Query 3: What is capitalism?


Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1


Result:  Capitalism is an economic system where the means of production, distribution, and exchange are privately owned and operated for profit.


Query 4: How is Aniruddh linked with Gary Vee?
Result:  Aniruddh is linked with Gary Vee through his attempt to invite him for the launch of E-Cell NFT, which turned out to be a scam.


